In [2]:
import sys, os
sys.path.append(os.pardir)

In [3]:
from pathlib import Path
import numpy as np
import polars as pl
import os
from hydra import initialize, compose

with initialize(config_path="../run/conf", version_base=None):
    cfg = compose("cv_train")

In [4]:
from src.utils.metrics import event_detection_ap
from src.utils.periodicity import get_periodicity_dict
from src.utils.common import trace
periodicity_dict = get_periodicity_dict(cfg)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
train_df = pl.read_parquet(Path(cfg.dir.data_dir) / "train_series.parquet")
train_df = train_df.with_columns(
            pl.col("timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%z")
        )

In [6]:
event_df = pl.read_csv(Path(cfg.dir.data_dir) / "train_events.csv").drop_nulls()
event_df = event_df.with_columns(
    pl.col("timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%z")
)

In [7]:
pred1_path2col_dict = {
   "148_gru_scale_factor.parquet": "148_gru_scale_factor",
    "156_gru_transformer_residual.parquet" : "156_gru_transformer_residual",
}
pred2_path2col_dict = {
    "../output/cv_inference/exp065_split_drop/single/train_pred.parquet": "exp065_split_drop",
    "../output/cv_inference/exp068_transformer/single/train_pred.parquet": "exp068_transformer",
    "../output/cv_inference/exp078_lstm/single/train_pred.parquet": "exp078_lstm",
}

In [8]:
df_list = [train_df]
events = ["onset", "wakeup"]
for path, name in pred1_path2col_dict.items():
    pred_df = pl.read_parquet(path)
    pred_df = pred_df.with_columns(
        [pl.col("prediction_"+col).cast(pl.Float32).alias(name+"_"+col) for col in events]
    ).select([name+"_"+col for col in events])
    df_list.append(pred_df)

for path, name in pred2_path2col_dict.items():
    pred_df = pl.read_parquet(path)
    pred_df = pred_df.with_columns(
        [pl.col("pred_"+col).cast(pl.Float32).alias(name+"_"+col) for col in events]
    ).select([name+"_"+col for col in events])
    df_list.append(pred_df)


pred_all_df = pl.concat(df_list, how="horizontal")
pred_all_df.head()

series_id,step,timestamp,anglez,enmo,148_gru_scale_factor_onset,148_gru_scale_factor_wakeup,156_gru_transformer_residual_onset,156_gru_transformer_residual_wakeup,exp065_split_drop_onset,exp065_split_drop_wakeup,exp068_transformer_onset,exp068_transformer_wakeup,exp078_lstm_onset,exp078_lstm_wakeup
str,u32,"datetime[μs, UTC]",f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""038441c925bb""",0,2018-08-14 19:30:00 UTC,2.6367,0.0217,1.3214e-9,0.000003,6.2617e-13,0.000028,0.014008,0.02298,0.000029,0.000035,0.004398,0.011375
"""038441c925bb""",1,2018-08-14 19:30:05 UTC,2.6368,0.0215,2.4195e-9,0.000003,9.6546e-13,0.000034,0.01252,0.018784,0.000031,0.000032,0.003567,0.009117
"""038441c925bb""",2,2018-08-14 19:30:10 UTC,2.637,0.0216,4.3217e-9,0.000004,1.1043e-12,0.000034,0.009552,0.010391,0.000033,0.000026,0.001902,0.004608
"""038441c925bb""",3,2018-08-14 19:30:15 UTC,2.6368,0.0213,4.5551e-9,0.000004,7.2700e-13,0.000034,0.009407,0.006439,0.000039,0.000022,0.001182,0.002462
"""038441c925bb""",4,2018-08-14 19:30:20 UTC,2.6368,0.0215,5.2894e-9,0.000003,7.1623e-13,0.000032,0.012085,0.006927,0.000048,0.000019,0.001407,0.002687


In [9]:
from scipy.optimize import minimize

In [10]:
from tqdm.auto import tqdm
from scipy.signal import find_peaks

def make_submission(
    preds_df: pl.DataFrame,
    periodicity_dict: dict[str, np.ndarray],
    height: float = 0.001,
    distance: int = 100,
    day_norm: bool = False,
    daily_score_offset: float = 1.0,
    pred_prefix: str = "prediction",
    late_date_rate: float | None = None,
) -> pl.DataFrame:
    event_dfs = []

    for series_id, series_df in tqdm(
        preds_df.group_by("series_id"), desc="find peaks", leave=False, total=len(preds_df["series_id"].unique())
    ):
        for event in ["onset", "wakeup"]:
            event_preds = series_df[f"{pred_prefix}_{event}"].to_numpy().copy()
            event_preds *= 1 - periodicity_dict[series_id][: len(event_preds)]
            steps = find_peaks(event_preds, height=height, distance=distance)[0]
            event_dfs.append(
                series_df.filter(pl.col("step").is_in(steps))
                .with_columns(pl.lit(event).alias("event"))
                .rename({f"{pred_prefix}_{event}": "score"})
                .select(["series_id", "step", "timestamp", "event", "score"])
            )

    submission_df = (
        pl.concat(event_dfs).sort(["series_id", "step"]).with_columns(pl.arange(0, pl.count()).alias("row_id"))
    )

    if day_norm:
        submission_df = submission_df.with_columns(
            pl.col("timestamp").dt.offset_by("2h").dt.date().alias("date")
        ).with_columns(
            pl.col("score") / (pl.col("score").sum().over(["series_id", "event", "date"]) + daily_score_offset)
        )

    if late_date_rate is not None:
        submission_df = (
            submission_df.with_columns(pl.col("timestamp").dt.offset_by("2h").dt.date().alias("date"))
            .with_columns(
                pl.col("date").min().over("series_id").alias("min_date"),
                pl.col("date").max().over("series_id").alias("max_date"),
            )
            .with_columns(
                pl.col("score")
                * (
                    1
                    - (
                        (1 - pl.lit(late_date_rate))
                        * (
                            (pl.col("date") - pl.col("min_date")).dt.days()
                            / ((pl.col("max_date") - pl.col("min_date")).dt.days() + 1.0)
                        )
                    )
                )
            )
        )

    return submission_df.select(["row_id", "series_id", "step", "event", "score"])


In [16]:
cols = ["148_gru_scale_factor", "156_gru_transformer_residual", "exp065_split_drop", 'exp068_transformer', "exp078_lstm"]

def calc_score(param, pred_all_df):
    param = param/param.sum()
    tmp_df = pred_all_df.with_columns(
        [(pl.col(f"{col}_{event}")* param[i]).alias(f'weighted_{col}_{event}') for i, col in enumerate(cols) for event in events]
    )
    tmp_df = tmp_df.with_columns(
        [pl.sum_horizontal([f'weighted_{col}_{event}' for col in cols]).alias(f"prediction_{event}")
        for event in events]
    )

    sub_df1 = make_submission(
        tmp_df,
        periodicity_dict= periodicity_dict,
        height = 0.001,
        distance = 107,
        daily_score_offset = 1.0,
        day_norm=True
    ) 
    score = event_detection_ap(
        event_df.to_pandas(),
        sub_df1.to_pandas(),
    )
    print(param, score)

    return -score

In [18]:
calc_score(np.array([0.42164168, 0.4814393,0,0,0]),pred_all_df)

find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

[0.46689244 0.53310756 0.         0.         0.        ] 0.816267170390424


-0.816267170390424

In [25]:

weights = [1.0/len(cols) for _ in range(len(cols)) ]
# cons = ({'type': 'eq', 'fun': lambda x: x.sum()})
res = minimize(calc_score, weights, args=(pred_all_df), 
               method='Nelder-Mead',
               options={'disp':True, "maxiter":100}, 
               bounds=[[0.0, 1.0] for _ in range(len(weights))]
              )
# [0.42164168 0.48143932 0.096919   0.         0.        ] 0.8197241628618371


find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

[0.2 0.2 0.2 0.2 0.2] 0.8142127037458327


find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

...

[0.43262263 0.47468035 0.09269702 0.         0.        ] 0.819695659577053


find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

[0.43619067 0.48223903 0.0815703  0.         0.        ] 0.8196486734982109


KeyboardInterrupt: 

In [33]:
cols = ["exp065_split_drop", 'exp068_transformer', "exp078_lstm"]

def calc_score(param, pred_all_df):
    param = param/param.sum()
    tmp_df = pred_all_df.with_columns(
        [(pl.col(f"{col}_{event}")* param[i]).alias(f'weighted_{col}_{event}') for i, col in enumerate(cols) for event in events]
    )
    tmp_df = tmp_df.with_columns(
        [pl.sum_horizontal([f'weighted_{col}_{event}' for col in cols]).alias(f"prediction_{event}")
        for event in events]
    )

    sub_df1 = make_submission(
        tmp_df,
        periodicity_dict= periodicity_dict,
        height = 0.001,
        distance = 107,
        daily_score_offset = 1.0,
        day_norm=False
    ) 
    score = event_detection_ap(
        event_df.to_pandas(),
        sub_df1.to_pandas(),
    )
    print(param, score)

    return -score



weights = [1.0/len(cols) for _ in range(len(cols)) ]
# cons = ({'type': 'eq', 'fun': lambda x: x.sum()})
res = minimize(calc_score, weights, args=(pred_all_df), 
               method='Nelder-Mead',
               options={'disp':True, "maxiter":100}, 
               bounds=[[0.0, 1.0] for _ in range(len(weights))]
              )
# 0.7966527549813314


find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

[0.33333333 0.33333333 0.33333333] 0.8023403512023469


find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

...

find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

[0.31948052 0.3360232  0.34449628] 0.8024224492237793


find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from tqdm.auto import tqdm
def score_ternary_search_distance(
    val_event_df: pl.DataFrame, pred_df, score_th: float = 0.005, end_diff: int=2, prefix="prediction"
) -> [float, float]:
    """
    post_process_for_seg のパラメータdistanceを ternary searchで探索する
    """
    l = 5
    r = 150
    cnt = 0
    best_score = 0.0
    best_distance = 0

    for cnt in tqdm(range(5)):
        if r - l < 1:
            break
        m1 = int(l + (r - l) / 3)
        m2 = int(r - (r - l) / 3)
        score1 = event_detection_ap(
            val_event_df.to_pandas(),
            make_submission(
                pred_df,
                height = score_th,
                distance = m1,
                periodicity_dict = periodicity_dict,
                prefix=prefix,
            ).to_pandas()
        )
        score2 = event_detection_ap(
            val_event_df.to_pandas(),
            make_submission(
                pred_df,
                height = score_th,
                distance = m2,
                periodicity_dict = periodicity_dict,
                prefix=prefix,
            ).to_pandas(),
        )

        if score1 >= score2:
            r = m2
            best_score = score1
            best_distance = m1

        else:
            l = m1
            best_score = score2
            best_distance = m2

        tqdm.write(f"score1(m1): {score1:.5f}({m1:.5f}), score2(m2): {score2:.5f}({m2:.5f}), l: {l:.5f}, r: {r:.5f}")

        if abs(m2 - m1) <= end_diff:
            break

    return best_score, best_distance


In [15]:
cols = ["148_gru_scale_factor", "156_gru_transformer_residual", "exp065_split_drop", 'exp068_transformer', "exp078_lstm"]

col = "exp065_split_drop"
tmp_df = pred_all_df.with_columns(
    [pl.col(f'{col}_{event}').alias(f"prediction_{event}") for event in events]
)

sub_df1 = make_submission(
    tmp_df,
    periodicity_dict= periodicity_dict,
    height = 0.001,
    distance = 107,
    daily_score_offset = 1.0,
    day_norm=True
) 
score = event_detection_ap(
    event_df.to_pandas(),
    sub_df1.to_pandas(),
)
print(score) #0.78818291 0.756548
 

find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

0.7565489802637723
